### Neural Network Model

#### Build an ANN model

In [ ]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [ ]:
TRAIN_DATA_PATH = '/content/sample_data/california_housing_train.csv'
TEST_DATA_PATH = '/content/sample_data/california_housing_test.csv'
TARGET_NAME = 'median_house_value'

df= pd.read_csv("")

#x values from dataframe, FEATURES
X= df.iloc[:, :]

#y values from dataframe, TARGET
y= df.iloc[:, :]

FileNotFoundError: ignored

In [ ]:
print(X.shape())
print(y.shape())

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(X, y, test_size=0.2)

In [ ]:
print(x_train.shape())
print(x_test.shape())
print(y_train.shape())
print(y_test.shape())

In [ ]:
"""
  Standard Scale test and train data
  Z - Score normalization
  """

  def scale_datasets():
    standard_scaler = StandardScaler()

    x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
    )

    x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
    )

  return x_train_scaled, x_test_scaled
  
x_train_scaled, x_test_scaled = scale_datasets(x_train, x_test)

In [ ]:
hidden_units1 = 160 #CAN CHANGE THEM, hyperparam search
hidden_units2 = 480
hidden_units3 = 256
learning_rate = 0.01

in_shape= # = to the number of input columns 

# Creating model using the Sequential in tensorflow

#DO WE USE FLATTEN??

def build_model():
  model = Sequential([
    Dense(hidden_units1, input_shape= (in_shape,), kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
    Dense(1, kernel_initializer='normal', activation='linear') #linear activation for regression 
  ])
  return model

#OR 
model=Sequential()
model.add(Dense(hidden_units1, input_shape= (in_shape,), activation='sigmoid'))

In [ ]:
# build the model
model = build_model()

In [ ]:
# loss function
msle = MeanSquaredLogarithmicError()

#sgd= SGD(lr=0.01) or ADAM
#TRY BOTH

model.compile(
    loss=msle, #loss= 'mean_squared_error'
    optimizer=Adam(learning_rate=learning_rate), #optimizer= sgd,
    metrics=[msle]
)

In [ ]:
model.summary()

In [ ]:
# train the model
history = model.fit(
    x_train_scaled.values, 
    y_train.values, 
    epochs=10, #10, 20, 100 ... ?
    batch_size=64, #32, 64, 128
    validation_split=0.2,
    verbose=1
)

In [ ]:
def plot_history(history, key):
  plt.plot(history.history[key])
  plt.plot(history.history['val_'+key])
  plt.xlabel("Epochs")
  plt.ylabel(key)
  plt.legend([key, 'val_'+key])
  plt.show()
# Plot the history
plot_history(history, 'mean_squared_logarithmic_error')

In [ ]:
# evaluate the model
y_pred= model.predict(x_test_scaled)
print('R^2: {:.5f}'.format(r2_score(ytest, ypred)))
print('RMSE: {:.5f}'.format(np.sqrt(mean_squared_error(ytest, ypred))))
print('MSE: {:.5f}'.format(mean_squared_error(ytest, ypred)))
print('MAE: {:.5f}'.format(mean_absolute_error(ytest, ypred)))

In [ ]:
# add the predictions to the dataframe
# x_test['prediction'] = model.predict(x_test_scaled)
x_test['prediction'] = y_pred

#### Build an RNN model

In [ ]:
#define the model

def build_rnn_model():
  model = Sequential([
  #LSTM layer 
  LSTM(64, return_sequences=False, dropout=0.1, recurrent_dropout=0.1)
  # Fully connected layer
  Dense(64, activation='relu')
  # Dropout for regularization
  Dropout(0.5)
  # Output layer
  Dense(1, activation='linear')
  ])
  return model

In [ ]:
# build the model
rnn_model = build_rnn_model()

In [ ]:
# loss function
msle = MeanSquaredLogarithmicError()

#sgd= SGD(lr=0.01) or ADAM
#TRY BOTH

rnn_model.compile(
    loss=msle, #loss= 'mean_squared_error'
    optimizer=Adam(learning_rate=learning_rate), #optimizer= sgd,
    metrics=[msle]
)

In [ ]:
rnn_model.summary()

In [ ]:
# train the model
history_rnn = rnn_model.fit(
    x_train_scaled.values, 
    y_train.values, 
    epochs=10, #10, 20, 100 ... ?
    batch_size=64, #32, 64, 128
    validation_split=0.2,
    verbose=1
)

In [ ]:
plot_history(history_rnn, 'mean_squared_logarithmic_error')

In [ ]:
# evaluate the model
y_pred= rnn_model.predict(x_test_scaled)
print('R^2: {:.5f}'.format(r2_score(ytest, ypred)))
print('RMSE: {:.5f}'.format(np.sqrt(mean_squared_error(ytest, ypred))))
print('MSE: {:.5f}'.format(mean_squared_error(ytest, ypred)))
print('MAE: {:.5f}'.format(mean_absolute_error(ytest, ypred)))

#### Define a model with hyperparameter tuning 

In [ ]:
def model_builder(hp):
  model = Sequential()

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))

  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mean_squared_error',
                metrics=['accuracy'])

  return model

In [ ]:
# instantiate the tuner and perform hypertuning 

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     #factor=3,
                     )

In [ ]:
#Create a callback to stop training early after reaching a certain value for the validation loss

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
# run the search 
tuner.search(x_train_scaled, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs

model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model

hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

https://www.analyticsvidhya.com/blog/2021/08/a-walk-through-of-regression-analysis-using-artificial-neural-networks-in-tensorflow/

https://www.tensorflow.org/tutorials/keras/keras_tuner

### SVM 

#### A Basic SVR for regression

In [ ]:
df = pd.read_csv('')
X = df.iloc[:, 1:2].values
y = df.iloc[:, 2].values

In [ ]:
from sklearn.preprocessing import StandardScaler

sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)

In [ ]:
from sklearn.svm import SVR

regressor = SVR(kernel = 'linear')
regressor.fit(X, y)

In [ ]:
y_pred = regressor.predict(6.5)
y_pred = sc_y.inverse_transform(y_pred) 

In [ ]:
X_grid = np.arange(min(X), max(X), 0.01) #this step required because data is feature scaled.
X_grid = X_grid.reshape((len(X_grid), 1))
plt.scatter(X, y, color = 'red')
plt.plot(X_grid, regressor.predict(X_grid), color = 'blue')
plt.title('')
plt.xlabel('')
plt.ylabel('')
plt.show()

https://www.analyticsvidhya.com/blog/2020/03/support-vector-regression-tutorial-for-machine-learning/


#### SVR Using Cross Validation and Hyperparameter Tuning

In [ ]:
import math
import itertools
import optunity
import optunity.metrics
import sklearn.svm

In [ ]:
# explicitly generate the outer_cv decorator so we can use it more than once

outer_cv = optunity.cross_validated(x=x_train, y=y_train, num_folds=10) #10-fold cros validation

def compute_mse_standard(x_train, y_train, x_test, y_test):
    """Computes MSE of an SVR with RBF kernel and default hyperparameters.
    """
    model = sklearn.svm.SVR().fit(x_train, y_train)
    predictions = model.predict(x_test)
    return optunity.metrics.mse(y_test, predictions)

# wrap with outer cross-validation
compute_mse_standard = outer_cv(compute_mse_standard) 

In [ ]:
compute_mse_standard() # returns a three-fold cross-validation estimate of MSE for an SVR with default hyperparameters.

In [ ]:
# function that returns MSE based on optimized hyperparameters, where we choose a polynomial kernel in advance.

def compute_mse_poly_tuned(x_train, y_train, x_test, y_test):
    """Computes MSE of an SVR with RBF kernel and optimized hyperparameters."""

    # define objective function for tuning
    @optunity.cross_validated(x=x_train, y=y_train, num_iter=2, num_folds=5)
    def tune_cv(x_train, y_train, x_test, y_test, C, degree, coef0):
        model = sklearn.svm.SVR(C=C, degree=degree, coef0=coef0, kernel='poly').fit(x_train, y_train)
        predictions = model.predict(x_test)
        return optunity.metrics.mse(y_test, predictions)

    # optimize parameters
    optimal_pars, _, _ = optunity.minimize(tune_cv, 150, C=[1000, 20000], degree=[2, 5], coef0=[0, 1])
    print("optimal hyperparameters: " + str(optimal_pars))

    tuned_model = sklearn.svm.SVR(kernel='poly', **optimal_pars).fit(x_train, y_train)
    predictions = tuned_model.predict(x_test)
    return optunity.metrics.mse(y_test, predictions)

# wrap with outer cross-validation
compute_mse_poly_tuned = outer_cv(compute_mse_poly_tuned)

In [ ]:
compute_mse_poly_tuned()

In [ ]:
# using an RBF kernel 

def compute_mse_rbf_tuned(x_train, y_train, x_test, y_test):
    """Computes MSE of an SVR with RBF kernel and optimized hyperparameters."""

    # define objective function for tuning
    @optunity.cross_validated(x=x_train, y=y_train, num_iter=2, num_folds=5)
    def tune_cv(x_train, y_train, x_test, y_test, C, gamma):
        model = sklearn.svm.SVR(C=C, gamma=gamma).fit(x_train, y_train)
        predictions = model.predict(x_test)
        return optunity.metrics.mse(y_test, predictions)

    # optimize parameters
    optimal_pars, _, _ = optunity.minimize(tune_cv, 150, C=[1, 100], gamma=[0, 50])
    print("optimal hyperparameters: " + str(optimal_pars))

    tuned_model = sklearn.svm.SVR(**optimal_pars).fit(x_train, y_train)
    predictions = tuned_model.predict(x_test)
    return optunity.metrics.mse(y_test, predictions)

# wrap with outer cross-validation
compute_mse_rbf_tuned = outer_cv(compute_mse_rbf_tuned)

In [ ]:
compute_mse_rbf_tuned()

We can do all prevoious tests using optunity:




In [ ]:
# we define the search space (we will try the linear, polynomial and RBF kernel).

space = {'kernel': {'linear': {'C': [0, 100]},
                    'rbf': {'gamma': [0, 50], 'C': [1, 100]},
                    'poly': {'degree': [2, 5], 'C': [1000, 20000], 'coef0': [0, 1]}
                    }
         }

In [ ]:
# we perform nested cross validation 

def compute_mse_all_tuned(x_train, y_train, x_test, y_test):
    """Computes MSE of an SVR with RBF kernel and optimized hyperparameters."""

    # define objective function for tuning
    @optunity.cross_validated(x=x_train, y=y_train, num_iter=2, num_folds=5)
    def tune_cv(x_train, y_train, x_test, y_test, kernel, C, gamma, degree, coef0):
        if kernel == 'linear':
            model = sklearn.svm.SVR(kernel=kernel, C=C)
        elif kernel == 'poly':
            model = sklearn.svm.SVR(kernel=kernel, C=C, degree=degree, coef0=coef0)
        elif kernel == 'rbf':
            model = sklearn.svm.SVR(kernel=kernel, C=C, gamma=gamma)
        else:
            raise ArgumentError("Unknown kernel function: %s" % kernel)
        model.fit(x_train, y_train)

        predictions = model.predict(x_test)
        return optunity.metrics.mse(y_test, predictions)

    # optimize parameters
    optimal_pars, _, _ = optunity.minimize_structured(tune_cv, num_evals=150, search_space=space)

    # remove hyperparameters with None value from optimal pars
    for k, v in optimal_pars.items():
        if v is None: del optimal_pars[k]
    print("optimal hyperparameters: " + str(optimal_pars))

    tuned_model = sklearn.svm.SVR(**optimal_pars).fit(x_train, y_train)
    predictions = tuned_model.predict(x_test)
    return optunity.metrics.mse(y_test, predictions)

# wrap with outer cross-validation
compute_mse_all_tuned = outer_cv(compute_mse_all_tuned)

In [ ]:
compute_mse_all_tuned() #get the optimal hyperparameter sets as outputs 

https://optunity.readthedocs.io/en/latest/notebooks/notebooks/sklearn-svr.html